In [3]:
import pandas
import folium
import json
import numpy as np
from IPython.display import HTML

#print(folium.__file__)
#print(folium.__version__)

ImportError: C extension: /projects/29e3300b-990d-41a4-b59c-c25ae7930627/.local/lib/python2.7/site-packages/pandas/hashtable.so: undefined symbol: PyFPE_jbuf not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext --inplace' to build the C extensions first.

In [ ]:
#Get House results data
url = 'http://electionresults.sos.state.mn.us/Results/MediaResult/20?mediafileid=20'
df = pd.read_csv(url, sep = ';', header = None)

df.head()


In [ ]:
#Looks wonky, lets clean it up.
#Column headings are available here: http://electionresults.sos.state.mn.us/Select/DownloadFileFormats/20
#Let's eliminate useless columns
df1 = df[[5, 7, 10, 13, 14, 15]]
#Rename to something useful
df1.columns = ['District', 'Candidate', 'Party', 'Votes_Cnt', 'Votes_Pct', 'Total_Voters']

#Eliminate write-ins
df1 = df1[df1['Party'] != 'WI']
#condense variables
data = df1[df1['Party'] == 'DFL']
data.to_json('data.json') 
data.reset_index()
#data.reset_index()

data.info()

In [ ]:
data.head()

In [ ]:
#Get shapefile
# Shapefile is here http://www.gis.leg.mn/html/download.html but I could not download it directly with Python - maybe I'll get there someday.
shape_url = 'https://raw.githubusercontent.com/Ccantey/LCC-DistrictFinder/master/data/HSE2012.json'
#map.geo_json(geo_path=state_geo, data=state_data,
           #  columns=['State', 'Unemployment'],
           #  threshold_scale=[5, 6, 7, 8, 9, 10],
            # key_on='feature.id',
            # fill_color='BuPu', fill_opacity=0.7, line_opacity=0.5,
            # legend_name='Unemployment Rate (%)',
            # reset=True)


import json
#District_geo1 = "L2012-1.json"
District_geo = shape_url
#District_geo = json.dumps(json.load(open(District_geo1, 'r')))

def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))

def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

def patcher(fn='map.html'):
    f=open(fn,'r')
    html=f.read()
    f.close()
    html=html.replace('"//','"http://')
    f=open(fn,'w')
    f.write(html)
    f.close()

from IPython.display import HTML

def display(m, height=500):
    """Takes a folium instance and embed HTML."""
    m._build_map()
    srcdoc = m.HTML.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{0}" '
                 'style="width: 100%; height: {1}px; '
                 'border: none"></iframe>'.format(srcdoc, height))
    return embed

#http://nbviewer.ipython.org/gist/psychemedia/fbcd7cf1daabe0004e27/folium_shapefiles.ipynb 
#Examples - http://blog.ouseful.info/2015/04/17/creating-interactive-election-maps-using-folium-and-ipython-notebooks/ 


MN_COORDS = [44.9543070,-93.1022220]
blank_map = folium.Map(location = MN_COORDS, zoom_start = 10)

blank_map.geo_json(geo_path = shape_url, 
              data_out="data1.json", 
              data=data, 
              columns=['District','Votes_Pct'], 
              key_on='feature.properties.district',
              #threshold_scale=[0, 40, 50, 65]
              fill_color = 'RdBu',
              fill_opacity=0.7, 
              line_opacity=0.9,
              legend_name = 'Vote Percentage',
              reset="True")
embed_map(blank_map)


#Test


#geojson_url_iw='http://mapit.mysociety.org/area/65791.geojson'
#iw_map = folium.Map(location=[50.666, -1.37], zoom_start=11)
#iw_map.geo_json(geo_path= geojson_url_iw)
#embed_map(iw_map)


#Add pop-ups for info to map